In [1]:
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install sentence_transformers
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 6.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00


In [2]:
import random
from typing import Dict, List, Union

In [3]:
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

In [4]:
llm_pipeline = pipeline(model='Qwen/Qwen2-7B-Instruct', device_map='auto', torch_dtype='auto')

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [5]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
def prepare_message_for_llm(text: Union[str, List[str]], categories: Dict[str, str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if len(categories) < 2:
        raise RuntimeError(f'The category list is too small! Expected 2 or more categories, got {len(categories)} ones.')
    categories_ = sorted(list(categories.keys()))
    categories_as_string = ', '.join(categories_[:-1]) + ' и ' + categories_[-1]
    if isinstance(text, str):
        prompt = f"Прочтите внимательно следующий текст и выберите одну из тем, к которой он больше всего относится." \
                 f"Выберите только одну тему из списка и ответьте кратко, указав только её название.\n\n" \
                 f"Список доступных тем: {categories_as_string}.\n\n" \
                 f'Текст: Созерцание цветения сакуры, называемое "ханами", вошло в японскую культуру еще в VIII веке.\nВаш ответ: entertainment\n' \
                 f'Текст: Спутники, каждый из которых был тяжелее 1 000 фунтов и перемещался со скоростью приблизительно 17 500 миль в час, столкнулись на высоте 491 мили над поверхностью Земли.\nВаш ответ: geography\n' \
                 f'Текст: Прочие варианты, которые основаны на биологических ритмах, включают в себя прием жидкости в больших количествах (в частности, воды или чая, известного мочегонного средства) перед сном, что заставляет человека вставать, чтобы помочиться.\nВаш ответ: health\n' \
                 f'Текст: Одним из наиболее заслуживающих внимания недавних примеров этого была компания в Северной Атлантике в ходе Второй мировой войны. Американцы пытались перевезти людей и материалы через Атлантический океан, чтобы помочь Британии.\nВаш ответ: politics\n' \
                 f'Текст: Для запуска в космос спутника или телескопа необходима гигантская ракета высотой более 100 футов.\nВаш ответ: science/technology\n' \
                 f'Текст: Окончательным счётом стала победа в одно очко, 21 к 20, что закончило победную серию All Black в 15-ти играх.\nВаш ответ: sports\n' \
                 f'Текст: В некоторых поездах, пересекающих границу, контроль осуществляется во время движения, поэтому, садясь на такой поезд, следует иметь при себе действительное удостоверение личности.\nВаш ответ: travel\n'
        prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
        messages = [
            {
                'role': 'system',
                'content': 'Вы - эксперт по классификации текстов на русском языке, глубоко анализируете содержание и определяете тему, к которой текст относится, строго на основе предложенного списка тем.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    else:
        messages = []
        for it in text:
            prompt = f"Прочтите внимательно следующий текст и выберите одну из тем, к которой он больше всего относится." \
                     f"Выберите только одну тему из списка и ответьте кратко, указав только её название.\n\n" \
                     f"Список доступных тем: {categories_as_string}.\n\n" \
                     f"Примеры:\n" \
                     f'Текст: Созерцание цветения сакуры, называемое "ханами", вошло в японскую культуру еще в VIII веке.\nВаш ответ: entertainment\n' \
                     f'Текст: Спутники, каждый из которых был тяжелее 1 000 фунтов и перемещался со скоростью приблизительно 17 500 миль в час, столкнулись на высоте 491 мили над поверхностью Земли.\nВаш ответ: geography\n' \
                     f'Текст: Прочие варианты, которые основаны на биологических ритмах, включают в себя прием жидкости в больших количествах (в частности, воды или чая, известного мочегонного средства) перед сном, что заставляет человека вставать, чтобы помочиться.\nВаш ответ: health\n' \
                     f'Текст: Одним из наиболее заслуживающих внимания недавних примеров этого была компания в Северной Атлантике в ходе Второй мировой войны. Американцы пытались перевезти людей и материалы через Атлантический океан, чтобы помочь Британии.\nВаш ответ: politics\n' \
                     f'Текст: Для запуска в космос спутника или телескопа необходима гигантская ракета высотой более 100 футов.\nВаш ответ: science/technology\n' \
                     f'Текст: Окончатель ным счётом стала победа в одно очко, 21 к 20, что закончило победную серию All Black в 15-ти играх.\nВаш ответ: sports\n' \
                     f'Текст: В некоторых поездах, пересекающих границу, контроль осуществляется во время движения, поэтому, садясь на такой поезд, следует иметь при себе действительное удостоверение личности.\nВаш ответ: travel\n'
            prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
            messages.append([
                {
                    'role': 'system',
                    'content': 'Вы - эксперт по классификации текстов на русском языке, глубоко анализируете содержание и определяете тему, к которой текст относится, строго на основе предложенного списка тем.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ])
    return {'message_for_llm': messages}

In [7]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))

examples_by_categories = dict()
for current_category in list_of_categories:
    examples_by_categories[current_category] = random.choice(
        train_set.filter(lambda it: it['category'] == current_category)['text']
    )

validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [8]:
class PostProcessor:
    def __init__(self, model_name='cde-small'):
        self.model_name = model_name
        if self.model_name == 'cde-small':
            self.model = SentenceTransformer("jxm/cde-small-v1", trust_remote_code=True)

    def process(self, true_cats, pred_cats):
        embs_dict = {cat : self.model.encode(cat) for cat in list_of_categories}
        preds = [x[-1]['content'] for x in y_pred]
        
        pred_processed = []
        for pred in pred_cats:
            res_cat = pred
            if pred not in embs_dict.keys():
                emb = self.model.encode(pred)
                max_dist = 0
                for cat in list_of_categories:
                    if cosine(emb, embs_dict[cat]) > max_dist:
                        max_dist = cosine(emb, embs_dict[cat])
                        res_cat = cat
            pred_processed.append(res_cat)

        return pred_processed

In [9]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
y_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [10]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                      precision    recall  f1-score   support

       entertainment       0.75      0.67      0.71         9
           geography       0.88      0.88      0.88         8
              health       0.82      0.82      0.82        11
             history       0.00      0.00      0.00         0
               music       0.00      0.00      0.00         0
            politics       0.86      0.86      0.86        14
    religion/culture       0.00      0.00      0.00         0
  science/technology       0.91      0.84      0.87        25
            security       0.00      0.00      0.00         0
              sports       0.92      1.00      0.96        12
technological change       0.00      0.00      0.00         0
             traffic       0.00      0.00      0.00         0
           transport       0.00      0.00      0.00         0
              travel       0.87      0.65      0.74        20

            accuracy                           0.81        99
      

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.75      0.67      0.71         9
         geography       0.88      0.88      0.88         8
            health       0.82      0.82      0.82        11
          politics       0.80      0.86      0.83        14
science/technology       0.92      0.88      0.90        25
            sports       0.75      1.00      0.86        12
            travel       0.82      0.70      0.76        20

          accuracy                           0.83        99
         macro avg       0.82      0.83      0.82        99
      weighted avg       0.83      0.83      0.83        99



In [12]:
pp = PostProcessor(model_name='cde-small')
y_pred_proc = pp.process(list_of_categories, [x[-1]['content'] for x in y_pred])
print(classification_report(y_true=y_true, y_pred=y_pred_proc))

modules.json:   0%|          | 0.00/149 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/276k [00:00<?, ?B/s]

sentence_transformers_impl.py:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jxm/cde-small-v1:
- sentence_transformers_impl.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

     entertainment       0.75      0.67      0.71         9
         geography       0.88      0.88      0.88         8
            health       0.75      0.82      0.78        11
          politics       0.86      0.86      0.86        14
science/technology       0.84      0.84      0.84        25
            sports       0.86      1.00      0.92        12
            travel       0.78      0.70      0.74        20

          accuracy                           0.82        99
         macro avg       0.82      0.82      0.82        99
      weighted avg       0.82      0.82      0.82        99



In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']

In [24]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

               art       0.00      0.00      0.00         0
     communication       0.00      0.00      0.00         0
     entertainment       0.78      0.74      0.76        19
         geography       0.79      0.88      0.83        17
            health       0.87      0.91      0.89        22
  immigration/visa       0.00      0.00      0.00         0
        literature       0.00      0.00      0.00         0
             media       0.00      0.00      0.00         0
             music       0.00      0.00      0.00         0
          politics       0.97      0.97      0.97        30
science/technology       0.92      0.94      0.93        51
            sports       0.88      0.84      0.86        25
    traffic/safety       0.00      0.00      0.00         0
         transport       0.00      0.00      0.00         0
            travel       1.00      0.62      0.77        40
      volunteering       0.00      0.00

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.70      0.74      0.72        19
         geography       0.79      0.88      0.83        17
            health       0.77      0.91      0.83        22
          politics       0.88      0.97      0.92        30
science/technology       0.92      0.94      0.93        51
            sports       0.75      0.84      0.79        25
            travel       0.96      0.62      0.76        40

          accuracy                           0.84       204
         macro avg       0.82      0.84      0.83       204
      weighted avg       0.85      0.84      0.84       204



In [20]:
pp = PostProcessor(model_name='cde-small')
y_pred_proc = pp.process(list_of_categories, [x[-1]['content'] for x in y_pred])
print(classification_report(y_true=y_true, y_pred=y_pred_proc))

pytorch_model.bin:   0%|          | 0.00/549M [00:00<?, ?B/s]

/root/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/c1b1fd7a715b8eb2e232d34593154ac782c98ac9/modeling_hf_nomic_bert.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this e

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

     entertainment       0.70      0.74      0.72        19
         geography       0.79      0.88      0.83        17
            health       0.83      0.91      0.87        22
          politics       0.97      0.97      0.97        30
science/technology       0.87      0.94      0.91        51
            sports       0.84      0.84      0.84        25
            travel       0.94      0.72      0.82        40

          accuracy                           0.86       204
         macro avg       0.85      0.86      0.85       204
      weighted avg       0.87      0.86      0.86       204

